In [8]:
import sage_data_client
import matplotlib.pyplot as plt
import pandas as pd
from metpy.calc import dewpoint_from_relative_humidity, wet_bulb_temperature
from metpy.units import units
from PIL import Image
import numpy as np
import datetime
import xarray as xr
import os

from matplotlib.dates import DateFormatter

In [12]:
aqt_global_ATMOS = {'conventions': "CF 1.10",
                  'WSN':'W0A4',
                   'site_ID' : "ATMOS",
                  'CAMS_tag' : "CMS-AQT-006",
                  'datastream' : "CMS_aqt580_ATMOS_a1",
                  'plugin' : "registry.sagecontinuum.org/jrobrien/waggle-aqt:0.*",
                  'datalevel' : "a1",
                  'latitude' : 41.7016264,
                  'longitude' : -87.9956515}


aqt_global_UIC = {'conventions': "CF 1.10",
              'WSN':'W096',
               'site_ID' : "UIC",
              'CAMS_tag' : "CMS-AQT-011",
              'datastream' : "CMS_aqt580_UIC_a1",
              'plugin' :  "registry.sagecontinuum.org/jrobrien/waggle-aqt:0.*",
              'datalevel' : "a1",
              'latitude' : 41.869407936,
              'longitude' : -87.645806251}


aqt_global_NEIU = {'conventions': "CF 1.10",
                   'site_ID' : "NEIU",
                  'CAMS_tag' : "CMS-AQT-001",
                  'datastream' : "CMS_aqt580_NEIU_a1",
                  'datalevel' : "a1",
                   'WSN' : 'W08D',
                   'plugin' : 'registry.sagecontinuum.org/jrobrien/waggle-aqt:0.*',
                  'latitude' : 41.9804526,
                  'longitude' : -87.7196038}

aqt_global_CSU = {'conventions': "CF 1.10",
                   'site_ID' : "CSU",
                  'CAMS_tag' : "CMS-AQT-002",
                  'datastream' : "CMS_aqt580_CSU_a1",
                  'datalevel' : "a1",
                   'WSN' : 'W08E',
                   'plugin' : 'registry.sagecontinuum.org/jrobrien/waggle-aqt:0.*',
                   'latitude' : 41.71991216,
                  'longitude' : -87.612834722}

aqt_global_NU = {'conventions': "CF 1.10",
                   'site_ID' : "NU",
                  'CAMS_tag' : "CMS-AQT-003",
                  'datastream' : "CMS_aqt580_NU_a1",
                  'datalevel' : "a1",
                   'WSN' : 'W099',
                   'plugin' : 'registry.sagecontinuum.org/jrobrien/waggle-aqt:0.*',
                    'latitude' : 42.051469749,
                  'longitude' : -87.677667183}



var_attrs_aqt = {'pm2.5' : {'standard_name' : 'mole_concentration_of_pm2p5_ambient_aerosol_particles_in_air',
                       'units' : 'ug/m^3'},
            'pm10.0' : {'standard_name' : 'mole_concentration_of_pm10p0_ambient_aerosol_particles_in_air',
                       'units' : 'ug/m^3'},
            'pm1.0' : {'standard_name' : 'mole_concentration_of_pm1p0_ambient_aerosol_particles_in_air',
                       'units' : 'ug/m^3'},
                'no' : {'standard_name' : 'mole_fraction_of_nitrogen_monoxide_in_air',
                       'units' : 'Parts Per Million'},
                'o3' : {'standard_name' : 'mole_fraction_of_ozone_in_air',
                       'units' : 'Parts Per Million'},
                'co' : {'standard_name' : 'mole_fraction_of_carbon_monoxide_in_air',
                       'units' : 'Parts Per Million'},
                'no2' : {'standard_name' : 'mole_fraction_of_nitrogen_dioxide_in_air',
                       'units' : 'Parts Per Million'},
                'temperature': {'standard_name' : 'air_temperature',
                       'units' : 'celsius'},
                'humidity': {'standard_name' : 'relative_humidity',
                       'units' : 'percent'},
                'dewpoint': {'standard_name' : 'dew_point_temperature',
                       'units' : 'celsius'},
                'pressure': {'standard_name' : 'air_pressure',
                       'units' : 'hPa'}}


In [13]:
def ingest_aqt(st, global_attrs, var_attrs):
    hours = 24
    start = st.strftime('%Y-%m-%dT%H:%M:%SZ')
    end = (st + datetime.timedelta(hours=hours)).strftime('%Y-%m-%dT%H:%M:%SZ')

    df_aq = sage_data_client.query(
        start=start,
        end=end, 
        filter={
            "plugin": global_attrs['plugin'],
            "vsn": global_attrs['WSN']
        }
    )

    pm25 = df_aq[df_aq['name']=='aqt.particle.pm2.5']
    pm10 = df_aq[df_aq['name']=='aqt.particle.pm1']
    pm100 = df_aq[df_aq['name']=='aqt.particle.pm10']

    no = df_aq[df_aq['name']=='aqt.gas.no']
    o3 = df_aq[df_aq['name']=='aqt.gas.ozone']
    no2 = df_aq[df_aq['name']=='aqt.gas.no2']
    co = df_aq[df_aq['name']=='aqt.gas.co']
    aqtemp = df_aq[df_aq['name']=='aqt.env.temp']
    aqhum = df_aq[df_aq['name']=='aqt.env.humidity']
    aqpres = df_aq[df_aq['name']=='aqt.env.pressure']


    pm25['time'] = pd.DatetimeIndex(pm25['timestamp'].values)

    aqvals = pm25.set_index('time')
    aqvals['pm2.5'] = aqvals.value.to_numpy().astype(float)
    aqvals['pm1.0'] = pm10.value.to_numpy().astype(float)
    aqvals['pm10.0'] = pm100.value.to_numpy().astype(float)

    aqvals['no'] = no.value.to_numpy().astype(float)
    aqvals['o3'] = o3.value.to_numpy().astype(float)
    aqvals['no2'] = no2.value.to_numpy().astype(float)
    aqvals['co'] = co.value.to_numpy().astype(float)
    aqvals['temperature'] =  aqtemp.value.to_numpy().astype(float)
    aqvals['humidity'] =  aqhum.value.to_numpy().astype(float)
    aqvals['pressure'] =  aqpres.value.to_numpy().astype(float)
    
    


    dp = dewpoint_from_relative_humidity( aqvals.temperature.to_numpy() * units.degC, 
                                         aqvals.humidity.to_numpy() * units.percent)

    aqvals['dewpoint'] = dp

    _ = aqvals.pop('value')
    _ = aqvals.pop('timestamp')
    
    end_fname = st.strftime('_%Y%m%d_%H%M%SZ.nc')
    start_fname = '/Users/scollis/data/aqt/CMS_AQT580_' + global_attrs['site_ID'] + '_' + global_attrs['datalevel']
    fname = start_fname + end_fname

    
    #fname = st.strftime('/Users/scollis/data/aqt/CMS_aqt580_NEIU_a1_%Y%m%d_%H%M%SZ.nc')
    valsxr = xr.Dataset.from_dataframe(aqvals)
    valsxr = valsxr.sortby('time')
    
    
    valsxr = valsxr.assign_attrs(global_attrs)
    
    for varname in var_attrs.keys():
        valsxr[varname] = valsxr[varname].assign_attrs(var_attrs[varname])
    
    try:
        os.remove(fname)
    except OSError:
        pass
    
    if valsxr['pm2.5'].shape[0] > 0:
        valsxr.to_netcdf(fname)
    else:
        print('not saving... no data')
    
    #return valsxr



In [22]:

ingest_aqt(datetime.datetime(2024,7,1), aqt_global_UIC, var_attrs_aqt)


/var/folders/f4/f3jp1t2x0hddhf4vz50gzlnc0000gn/T/ipykernel_31031/3766339791.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pm25['time'] = pd.DatetimeIndex(pm25['timestamp'].values)


In [4]:
start_date = datetime.datetime(2024,5,10)
for i in range(13):
    this_date = start_date + datetime.timedelta(days=i)
    print(this_date)
    try:
        ingest_aqt(this_date, aqt_global_NU, var_attrs_aqt)
        print("Succeed")
    except:
        print("Fail")

NameError: name 'datetime' is not defined

In [5]:
t13 = ingest_aqt(datetime.datetime(2023,6,13))

NameError: name 'datetime' is not defined

In [ ]:
t14 = ingest_aqt(datetime.datetime(2023,6,14))

In [ ]:
! rm /Users/scollis/data/aqt/CMS_aqt580_NEIU_a1_20230614_000000Z.nc

In [ ]:
bob = t14['pm1.0']

In [ ]:
t14['pm1.0'] = t14['pm1.0'].assign_attrs({'units': 'bananas'})

In [ ]:
t14['pm1.0'].assign_attrs?

In [ ]:
t14

In [ ]:
t13['hum'].plot()

:command_line = "sonde_ingest -s sgp -f C1" ;
		:Conventions = "ARM-1.2" ;
		:process_version = "ingest-sonde-10.14-1.el7" ;
		:dod_version = "sondewnpn-b1-2.3" ;
		:input_source = "/data/collection/sgp/sgpsondeC1.00/good.2023APR221134.parsed" ;
		:site_id = "sgp" ;
		:platform_id = "sondewnpn" ;
		:facility_id = "C1" ;
		:data_level = "b1" ;
		:location_description = "Southern Great Plains (SGP), Lamont, Oklahoma" ;
		:datastream = "sgpsondewnpnC1.b1" ;
		:sample_int = "1.2 seconds" ;
		:averaging_int = "None" ;
		:serial_number = "T2710383" ;
		:ingest_software = "ingest-sonde-10.14-1.el7" ;